In [1]:
from kedmi.models.generator import *
from kedmi.models.discri import *
from kedmi.utils.helper import *
import torch
import os
import numpy as np
from kedmi.utils.kedmi_attack import mnist_inversion, dist_inversion

### load specified configuration and specify environment variables

In [2]:
CONFIG_PATH = "config/kedmi/config/celeba/attacking/celeba_cust.json"
configuration = load_json(json_file=CONFIG_PATH)

save_folder = os.path.join(
    f"{configuration['dataset']['name']}_{configuration['dataset']['model_name']}",
    configuration["attack"]["variant"],
)
prefix = os.path.join(
    os.path.join(configuration["root_path"], "kedmi_300ids"), save_folder
)
save_dir = os.path.join(prefix, "latent")
save_img_dir = os.path.join(
    prefix, "imgs_{}".format(configuration["attack"]["variant"])
)

os.makedirs(prefix, exist_ok=True)
os.makedirs(save_img_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(configuration["dataset"]["p_reg_path"], exist_ok=True)

### load trained models

In [3]:
targetnets, E, G, D, n_classes, fea_mean, fea_logvar = get_attack_model(configuration)

['VGG16', 'VGG16']
path_G checkpoints/kedmi/GAN/celeba/VGG16/improved_celeba_G.tar
path_D checkpoints/kedmi/GAN/celeba/VGG16/improved_celeba_D.tar


/home/bot/coding/bachelorarbeit/ba_code/kedmi/models/discri.py:79: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(self.T, 0, 1)


0
VGG16
Load classifier VGG16 at checkpoints/VGG16/nn_VGG16celeba.tar


/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1
VGG16
Load classifier VGG16 at checkpoints/VGG16/nn_VGG16celeba.tar


### set necessary params

In [4]:
N = 5
bs = 60

### Run attack

In [5]:
# Begin attacking
for i in range(1):
    iden = torch.from_numpy(np.arange(bs))

    # evaluate on the first 300 identities only
    target_cosines = 0
    eval_cosines = 0
    for idx in range(5):
        iden = iden % n_classes
        print(
            "--------------------- Attack batch [%s]------------------------------"
            % idx
        )
        print("Iden:{}".format(iden))
        save_dir_z = "{}/{}_{}".format(save_dir, i, idx)
        print("kedmi")

        dist_inversion(
            G,
            D,
            targetnets,
            E,
            iden,
            lr=configuration["attack"]["lr"],
            iter_times=configuration["attack"]["iters_mi"],
            momentum=0.9,
            lamda=100,
            clip_range=1,
            improved=configuration['attack']['improved_flag'],
            num_seeds=configuration['attack']['num_seeds'],
            used_loss=configuration['attack']['loss'],
            prefix=save_dir_z,
            save_img_dir=os.path.join(save_img_dir, "{}_".format(idx)),
            fea_mean=fea_mean,
            fea_logvar=fea_logvar,
            lam=configuration["attack"]["lam"],
            clipz=configuration['attack']['clipz'],
        )
        iden = iden + bs

--------------------- Attack batch [0]------------------------------
Iden:tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])
kedmi
criterion:logit_loss


/home/bot/.local/lib/python3.10/site-packages/torch/nn/parallel/comm.py:227: UserWarning: Using -1 to represent CPU tensor is deprecated. Please use a device object or string instead, e.g., "cpu".
  warnings.warn(


Iteration:300	Prior Loss:5.18	Iden Loss:-9.43	Attack Acc:28.33
Iteration:600	Prior Loss:4.73	Iden Loss:-15.54	Attack Acc:91.67
Iteration:900	Prior Loss:4.64	Iden Loss:-19.63	Attack Acc:100.00
Iteration:1200	Prior Loss:4.53	Iden Loss:-22.25	Attack Acc:100.00
Iteration:1500	Prior Loss:4.48	Iden Loss:-23.91	Attack Acc:100.00
Iteration:1800	Prior Loss:4.43	Iden Loss:-24.66	Attack Acc:100.00
Gesamtzeit für 2000 Iterationen: 232.00 Sekunden
--------------------- Attack batch [1]------------------------------
Iden:tensor([ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
         74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119])
kedmi
criterion:logit_loss
Iteration:300	Prior Loss:5.25	Iden Loss:-9.31	Attack Acc:25.00
Iteration:600	Prior Loss:4.87	Iden Loss:-16.51	Attack Ac